In [2]:
import numpy as np
import torch
import cv2 as cv
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from torch.optim import RMSprop
from torch.optim.lr_scheduler import StepLR
from data_loaders import CAERSDataset
from utils import get_transform, accuracy, accuracy_julia
from model import model_generator
from tqdm import tqdm
import os.path as osp
from keras.models import load_model
import tensorflow as tf
import os
import matplotlib.pyplot as plt

c:\Users\jdts\anaconda3\envs\emotionram\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:

captura = cv.VideoCapture(0)
model_path = '.\models\\faces_19_95.01295733262594.pth'
model = torch.load(model_path)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") ## Se a maquina não tiver GPU o teste será rodado na CPU


model.to(device)

opt = RMSprop(model.parameters(), lr=3e-3)
lr_scheduler = StepLR(opt, 60, 0.4)



: 

In [4]:
def set_emotion(label):

    if label == '0' :
        emotion = 'Angry'
    elif label == '1':
        emotion = 'Disgust'
    elif label == '2':
        emotion = 'Fear'
    elif label == '3':
        emotion = 'Happy'
    elif label == '4':
        emotion = 'Neutral'
    elif label == '5':
        emotion = 'Sad'
    elif label == '6':
        emotion = 'Surprise'
    return emotion

In [6]:

while(1):

    ret, frame = captura.read()
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    face_cascade = cv.CascadeClassifier('haarcascade_frontalface_alt.xml')
    faces = face_cascade.detectMultiScale(gray, 1.1, 2)


    

    print('Number of detected faces:', len(faces))\

        # loop over all the detected faces
    for (x,y,w,h) in faces:
        ret, frame = captura.read()
        emotion= []

        #To predict emotions
        my_transforms = get_transform(train=False)
        data = {
            'face': frame
            }
        transformed_frame = my_transforms(data)
        frame_torch = torch.asarray(transformed_frame["face"])
        frame_torch = torch.unsqueeze(frame_torch, 0)
        preds= model(frame_torch.to(device))
        preds = preds.to('cpu')
        emotion.extend(torch.argmax(preds, dim=1))
        emotion=str(emotion[0].numpy())
        em_pred= set_emotion(emotion)

          
        # To draw a rectangle around the detected face  
        face_cut =  frame[y:y+h, x: x+w]
        image= cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        cv.putText(image,em_pred, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        # Display an image in a window
        cv.imshow('emotionRAM',image)

        k = cv.waitKey(30) & 0xff
        if k == 27:
           break

captura.release()
cv.destroyAllWindows()

Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 1
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 2
Number of detected faces: 3
Number of detected faces: 1
Number of detected faces: 2
Number of detected faces: 1
Number of detected faces: 0
Number of detected faces: 2
Number of detected faces: 1
Number of detected f

KeyboardInterrupt: 